In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Load IMDB dataset
max_features = 20000  # number of words
maxlen = 200          # cut reviews after 200 words

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

def build_model(rnn_layer):
    model = keras.Sequential([
        layers.Embedding(max_features, 128),
        rnn_layer,
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

results = {}

# Try different recurrent layers
for rnn_layer in [
    layers.SimpleRNN(64),
    layers.GRU(64),
    layers.LSTM(64),
]:
    print(f"\nTraining with {rnn_layer.__class__.__name__}...")
    model = build_model(rnn_layer)
    history = model.fit(
        x_train, y_train,
        batch_size=64,
        epochs=2,          # keep small for runtime
        validation_split=0.2,
        verbose=2
    )
    score = model.evaluate(x_test, y_test, verbose=0)
    results[rnn_layer.__class__.__name__] = score[1]

print("\nFinal Accuracy Comparison:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")


17464789/17464789 [==============================] - 17s 1us/step

Training with SimpleRNN...
Epoch 1/2
313/313 - 33s - loss: 0.5065 - accuracy: 0.7375 - val_loss: 0.3732 - val_accuracy: 0.8442 - 33s/epoch - 104ms/step
Epoch 2/2
313/313 - 31s - loss: 0.3254 - accuracy: 0.8607 - val_loss: 0.4267 - val_accuracy: 0.8254 - 31s/epoch - 98ms/step

Training with GRU...
Epoch 1/2
313/313 - 68s - loss: 0.4572 - accuracy: 0.7686 - val_loss: 0.3611 - val_accuracy: 0.8422 - 68s/epoch - 216ms/step
Epoch 2/2
313/313 - 64s - loss: 0.2105 - accuracy: 0.9204 - val_loss: 0.3243 - val_accuracy: 0.8672 - 64s/epoch - 206ms/step

Training with LSTM...
Epoch 1/2
313/313 - 75s - loss: 0.4066 - accuracy: 0.8095 - val_loss: 0.3050 - val_accuracy: 0.8702 - 75s/epoch - 239ms/step
Epoch 2/2
313/313 - 73s - loss: 0.2022 - accuracy: 0.9240 - val_loss: 0.3118 - val_accuracy: 0.8730 - 73s/epoch - 232ms/step

Final Accuracy Comparison:
SimpleRNN: 0.8242
GRU: 0.8511
LSTM: 0.8649


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv3D

# Example ConvLSTM2D model
model = Sequential([
    ConvLSTM2D(filters=40, kernel_size=(3, 3), input_shape=(None, 40, 40, 1), padding="same", return_sequences=True),
    BatchNormalization(),
    ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True),
    BatchNormalization(),
    Conv3D(filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same")
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, None, 40, 40, 40   59200     
                             )                                   
                                                                 
 batch_normalization (Batch  (None, None, 40, 40, 40   160       
 Normalization)              )                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, None, 40, 40, 40   115360    
                             )                                   
                                                                 
 batch_normalization_1 (Bat  (None, None, 40, 40, 40   160       
 chNormalization)            )                                   
                                                                 
 conv3d (Conv3D)             (None, None, 40, 40, 1)  

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters
max_features = 10000  # Number of words to keep
maxlen = 500          # Cut texts after 500 words

# Load Reuters dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.reuters.load_data(num_words=max_features)

# Pad sequences so all have same length
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

# Convert labels to categorical
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

2110848/2110848 [==============================] - 1s 0us/step


In [5]:
def build_lstm():
    model = keras.Sequential([ layers.Embedding(max_features, 128, input_length=maxlen), layers.LSTM(64), layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(loss="categorical_crossentropy", optimizer="adam",  metrics=["accuracy"])
    return model

In [6]:
def build_conv1d():
    model = keras.Sequential([
        layers.Embedding(max_features, 128, input_length=maxlen),
        layers.Conv1D(64, 5, activation="relu"),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(64, 5, activation="relu"),
        layers.GlobalMaxPooling1D(),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [7]:
results = {}

# Train LSTM
print("\nTraining LSTM...")
lstm = build_lstm()
lstm.fit(x_train, y_train,  batch_size=64, epochs=3,  validation_split=0.2, verbose=2)
score = lstm.evaluate(x_test, y_test, verbose=0)
results["LSTM"] = score[1]

# Train Conv1D
print("\nTraining Conv1D...")
conv = build_conv1d()
conv.fit(x_train, y_train, batch_size=64, epochs=3, validation_split=0.2, verbose=2)
score = conv.evaluate(x_test, y_test, verbose=0)
results["Conv1D"] = score[1]

print("\nFinal Test Accuracy Comparison:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")


Training LSTM...
Epoch 1/3
113/113 - 69s - loss: 2.4321 - accuracy: 0.4274 - val_loss: 2.0324 - val_accuracy: 0.4841 - 69s/epoch - 608ms/step
Epoch 2/3
113/113 - 68s - loss: 1.8240 - accuracy: 0.5254 - val_loss: 1.6752 - val_accuracy: 0.5804 - 68s/epoch - 605ms/step
Epoch 3/3
113/113 - 75s - loss: 1.6605 - accuracy: 0.5550 - val_loss: 1.6487 - val_accuracy: 0.5732 - 75s/epoch - 664ms/step

Training Conv1D...
Epoch 1/3
113/113 - 37s - loss: 2.3251 - accuracy: 0.4525 - val_loss: 1.8283 - val_accuracy: 0.5264 - 37s/epoch - 326ms/step
Epoch 2/3
113/113 - 35s - loss: 1.6585 - accuracy: 0.5797 - val_loss: 1.5426 - val_accuracy: 0.6155 - 35s/epoch - 306ms/step
Epoch 3/3
113/113 - 34s - loss: 1.3504 - accuracy: 0.6786 - val_loss: 1.3348 - val_accuracy: 0.6917 - 34s/epoch - 304ms/step

Final Test Accuracy Comparison:
LSTM: 0.5726
Conv1D: 0.6754


In [12]:
import os
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

# Download dataset
fname = keras.utils.get_file("jena_climate.csv.zip", url, extract=True)


# Load into dataframe
df = pd.read_csv(fname)

print(df.head())

             Date Time  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
0  01.01.2009 00:10:00    996.52     -8.02    265.40        -8.90    93.3   
1  01.01.2009 00:20:00    996.57     -8.41    265.01        -9.28    93.4   
2  01.01.2009 00:30:00    996.53     -8.51    264.91        -9.31    93.9   
3  01.01.2009 00:40:00    996.51     -8.31    265.12        -9.07    94.2   
4  01.01.2009 00:50:00    996.51     -8.27    265.15        -9.04    94.1   

   VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  H2OC (mmol/mol)  \
0          3.33          3.11          0.22       1.94             3.12   
1          3.23          3.02          0.21       1.89             3.03   
2          3.21          3.01          0.20       1.88             3.02   
3          3.26          3.07          0.19       1.92             3.08   
4          3.27          3.08          0.19       1.92             3.09   

   rho (g/m**3)  wv (m/s)  max. wv (m/s)  wd (deg)  
0       1307.75      1.03        

In [13]:
# Convert dataframe to numpy (excluding the 'Date Time' column)
data = df.iloc[:, 1:].values

# Standardize (normalize)
mean = data[:200000].mean(axis=0)
std = data[:200000].std(axis=0)
data = (data - mean) / std

In [14]:
# Parameters
lookback = 720     # how many past timesteps (10 min * 720 = 5 days)
step = 6           # sample one data point every hour
delay = 144        # predict 24 hours in the future
batch_size = 128

def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while True:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]  # column 1 = temp in dataset
        yield samples, targets

In [15]:
train_gen = generator(data, lookback=lookback, delay=delay, min_index=0, max_index=200000, shuffle=True, step=step, batch_size=batch_size)

val_gen = generator(data, lookback=lookback, delay=delay, min_index=200001, max_index=300000, step=step, batch_size=batch_size)

test_gen = generator(data, lookback=lookback, delay=delay, min_index=300001, max_index=None, step=step, batch_size=batch_size)

val_steps = (300000 - 200001 - lookback) // batch_size
test_steps = (len(data) - 300001 - lookback) // batch_size

In [16]:
model = keras.Sequential([layers.LSTM(32, input_shape=(lookback // step, data.shape[-1])), layers.Dense(1)
])

model.compile(optimizer="rmsprop", loss="mae")

In [17]:
history = model.fit(train_gen, steps_per_epoch=500, epochs=20, validation_data=val_gen, validation_steps=val_steps)

     

Epoch 1/20
500/500 [==============================] - 55s 104ms/step - loss: 0.3039 - val_loss: 0.2746
Epoch 2/20
500/500 [==============================] - 52s 104ms/step - loss: 0.2778 - val_loss: 0.2685
Epoch 3/20
500/500 [==============================] - 52s 104ms/step - loss: 0.2653 - val_loss: 0.2715
Epoch 4/20
500/500 [==============================] - 52s 104ms/step - loss: 0.2563 - val_loss: 0.2749
Epoch 5/20
500/500 [==============================] - 54s 107ms/step - loss: 0.2469 - val_loss: 0.2774
Epoch 6/20
500/500 [==============================] - 53s 106ms/step - loss: 0.2404 - val_loss: 0.2783
Epoch 7/20
500/500 [==============================] - 53s 106ms/step - loss: 0.2333 - val_loss: 0.2850
Epoch 8/20
500/500 [==============================] - 53s 107ms/step - loss: 0.2289 - val_loss: 0.2841
Epoch 9/20
500/500 [==============================] - 53s 107ms/step - loss: 0.2249 - val_loss: 0.2898
Epoch 10/20
500/500 [==============================] - 56s 113ms/step - l

In [18]:
test_loss = model.evaluate(test_gen, steps=test_steps)
print("Test MAE:", test_loss)

936/936 [==============================] - 29s 31ms/step - loss: 0.3366
Test MAE: 0.33657678961753845
